In [30]:
# Librerias
import pandas as pd # Guardar informacion en dataframes
import glob # Buscar un patron de un archivo

# *T. cruzi* strain CL Brener STRINGdb

In [31]:
# Cargar el archivo CSV con manejo de errores
def cargar_csv(ruta):
    try:
        df = pd.read_csv(ruta)
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo CSV.")
        return None

# Guardar genes agrupados por pathways en un solo archivo
def guardar_genes_por_pathway(df, salida):
    if df is not None:
        df_filtrado = df[['description', 'category', 'term', 'inputGenes', 'preferredNames']]
        
        try:
            # Leer el archivo existente solo una vez al principio
            df_existente = pd.read_csv(salida)
            # Eliminar duplicados antes de agregar nuevas filas
            df_completo = pd.concat([df_existente, df_filtrado], ignore_index=True).drop_duplicates(subset=['description', 'category', 'term', 'inputGenes', 'preferredNames'])
        except FileNotFoundError:
            # Si no existe, crear uno nuevo con la información filtrada
            df_completo = df_filtrado
        
        df_completo.to_csv(salida, index=False)
    else:
        print("No se pudo guardar el archivo porque hubo un error al cargar los datos.")

# Cargar y procesar los archivos CSV
rutas_csv = [
    "ShinyGo/Patogeno/STRING_enrichmentKEGG_TcruzistrainCLBrenerSTRINGdb.csv",  # Segundo archivo
]

salida_csv = "ShinyGo/Patogeno/generate_TcruzistrainCLBrenerSTRINGdb/genes_pathway_TcruzistrainCLBrenerSTRINGdb.csv"  # Archivo de salida único

# Procesar ambos archivos
for ruta in rutas_csv:
    df = cargar_csv(ruta)
    guardar_genes_por_pathway(df, salida_csv)

In [32]:
# Cargar el archivo CSV
df = pd.read_csv("ShinyGo/Patogeno/generate_TcruzistrainCLBrenerSTRINGdb/genes_pathway_TcruzistrainCLBrenerSTRINGdb.csv")

# Separar los genes en filas y guardar en archivos TSV por pathway
for index, row in df.iterrows():
    input_genes = row['inputGenes'].split(',')
    preferred_names = row['preferredNames'].split(',')
    
    # Crear un DataFrame con las columnas inputGenes y preferredNames
    df_genes = pd.DataFrame({
        'inputGenes': input_genes,
        'preferredNames': preferred_names
    })
    
    # Crear el nombre del archivo basado en la descripción de la ruta (pathway)
    pathway_name = row['description'].replace(" ", "_")  # Reemplazar espacios por guiones bajos
    pathway_name = pathway_name.replace("/", "_")  # Reemplazar las barras "/" por guiones bajos
    pathway_name = pathway_name.replace("\\", "_")  # Reemplazar las barras invertidas "\\" por guiones bajos
    pathway_name = pathway_name.replace(":", "_")  # Reemplazar los dos puntos ":" por guiones bajos
    pathway_name = pathway_name.replace("-", "_")  # Reemplazar los dos puntos "-" por guiones bajos
    
    archivo_salida = f"ShinyGo/Patogeno/pathway_TcruzistrainCLBrenerSTRINGdb/{pathway_name}.tsv"
    
    # Guardar el DataFrame en un archivo TSV
    df_genes.to_csv(archivo_salida, sep='\t', index=False)

In [33]:
# Cargar el archivo CSV
df = pd.read_csv("ShinyGo/Patogeno/geneInfo_TcruzistrainCLBrenerSTRINGdb.csv")

# Seleccionar las columnas relevantes
df_selected = df[["User_input", "STRINGdb ID"]]

# Guardar en un archivo .tsv
df_selected.to_csv("ShinyGo/Patogeno/generate_TcruzistrainCLBrenerSTRINGdb/geneInfo_TcruzistrainCLBrenerSTRINGdb.tsv", sep='\t', index=False)

In [34]:
# Cargar el archivo geneInfo.tsv
gene_info = pd.read_csv("ShinyGo/Patogeno/generate_TcruzistrainCLBrenerSTRINGdb/geneInfo_TcruzistrainCLBrenerSTRINGdb.tsv", sep='\t')

# Usar glob para obtener todos los archivos .tsv en la carpeta de vías metabólicas
pathways_files = glob.glob("ShinyGo/Patogeno/pathway_TcruzistrainCLBrenerSTRINGdb/*.tsv")

# Crear un DataFrame vacío para almacenar las filas que coinciden
matching_rows = pd.DataFrame()

# Comparar STRINGdb ID con preferredNames en cada archivo de vías
for file in pathways_files:
    # Cargar el archivo de la vía metabólica
    pathway_data = pd.read_csv(file, sep='\t')
    
    # Verificar si hay coincidencias entre STRINGdb ID y preferredNames
    merged_data = pd.merge(gene_info[['User_input', 'STRINGdb ID']], pathway_data[['preferredNames']], 
                           left_on='STRINGdb ID', right_on='preferredNames', how='inner')
    
    # Si hay coincidencias, agregar las columnas necesarias a matching_rows
    if not merged_data.empty:
        matching_rows = pd.concat([matching_rows, merged_data[['User_input']]])

# Renombrar la columna 'User_input' a 'Protein_ID'
matching_rows = matching_rows.rename(columns={'User_input': 'Protein_ID'})

# Guardar el archivo final
matching_rows.to_csv("ShinyGo/Patogeno/generate_TcruzistrainCLBrenerSTRINGdb/PatogenoBase_TcruzistrainCLBrenerSTRINGdb.tsv", sep='\t', index=False)

# *T. cruzi* STRINGdb

In [35]:
# Cargar el archivo CSV con manejo de errores
def cargar_csv(ruta):
    try:
        df = pd.read_csv(ruta)
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo CSV.")
        return None

# Guardar genes agrupados por pathways en un solo archivo
def guardar_genes_por_pathway(df, salida):
    if df is not None:
        df_filtrado = df[['description', 'category', 'term', 'inputGenes', 'preferredNames']]
        
        try:
            # Leer el archivo existente solo una vez al principio
            df_existente = pd.read_csv(salida)
            # Eliminar duplicados antes de agregar nuevas filas
            df_completo = pd.concat([df_existente, df_filtrado], ignore_index=True).drop_duplicates(subset=['description', 'category', 'term', 'inputGenes', 'preferredNames'])
        except FileNotFoundError:
            # Si no existe, crear uno nuevo con la información filtrada
            df_completo = df_filtrado
        
        df_completo.to_csv(salida, index=False)
    else:
        print("No se pudo guardar el archivo porque hubo un error al cargar los datos.")

# Cargar y procesar los archivos CSV
rutas_csv = [
    "ShinyGo/Patogeno/STRING_enrichmentInterPro_TcruziSTRINGdb.csv",  # Reemplazar con el nombre real del primer archivo
    "ShinyGo/Patogeno/STRING_enrichmentPfamTcruziSTRINGdb.csv"
]

salida_csv = "ShinyGo/Patogeno/generate_TcruziSTRINGdb/genes_pathway_TcruziSTRINGdb.csv"  # Archivo de salida único

# Procesar ambos archivos
for ruta in rutas_csv:
    df = cargar_csv(ruta)
    guardar_genes_por_pathway(df, salida_csv)

In [36]:
# Cargar el archivo CSV
df = pd.read_csv("ShinyGo/Patogeno/generate_TcruziSTRINGdb/genes_pathway_TcruziSTRINGdb.csv")

# Separar los genes en filas y guardar en archivos TSV por pathway
for index, row in df.iterrows():
    input_genes = row['inputGenes'].split(',')
    preferred_names = row['preferredNames'].split(',')
    
    # Crear un DataFrame con las columnas inputGenes y preferredNames
    df_genes = pd.DataFrame({
        'inputGenes': input_genes,
        'preferredNames': preferred_names
    })
    
    # Crear el nombre del archivo basado en la descripción de la ruta (pathway)
    pathway_name = row['description'].replace(" ", "_")  # Reemplazar espacios por guiones bajos
    pathway_name = pathway_name.replace("/", "_")  # Reemplazar las barras "/" por guiones bajos
    pathway_name = pathway_name.replace("\\", "_")  # Reemplazar las barras invertidas "\\" por guiones bajos
    pathway_name = pathway_name.replace(":", "_")  # Reemplazar los dos puntos ":" por guiones bajos
    pathway_name = pathway_name.replace("-", "_")  # Reemplazar los dos puntos "-" por guiones bajos
    
    archivo_salida = f"ShinyGo/Patogeno/pathway_TcruziSTRINGdb/{pathway_name}.tsv"
    
    # Guardar el DataFrame en un archivo TSV
    df_genes.to_csv(archivo_salida, sep='\t', index=False)

In [37]:
# Cargar el archivo CSV
df = pd.read_csv("ShinyGo/Patogeno/geneInfo_TcruziSTRINGdb.csv")

# Seleccionar las columnas relevantes
df_selected = df[["User_input", "STRINGdb ID"]]

# Guardar en un archivo .tsv
df_selected.to_csv("ShinyGo/Patogeno/generate_TcruziSTRINGdb/geneInfo_TcruziSTRINGdb.tsv", sep='\t', index=False)

In [38]:
# Cargar el archivo geneInfo.tsv
gene_info = pd.read_csv("ShinyGo/Patogeno/generate_TcruziSTRINGdb/geneInfo_TcruziSTRINGdb.tsv", sep='\t')

# Usar glob para obtener todos los archivos .tsv en la carpeta de vías metabólicas
pathways_files = glob.glob("ShinyGo/Patogeno/pathway_TcruziSTRINGdb/*.tsv")

# Crear un DataFrame vacío para almacenar las filas que coinciden
matching_rows = pd.DataFrame()

# Comparar STRINGdb ID con preferredNames en cada archivo de vías
for file in pathways_files:
    # Cargar el archivo de la vía metabólica
    pathway_data = pd.read_csv(file, sep='\t')
    
    # Verificar si hay coincidencias entre STRINGdb ID y preferredNames
    merged_data = pd.merge(gene_info[['User_input', 'STRINGdb ID']], pathway_data[['preferredNames']], 
                           left_on='STRINGdb ID', right_on='preferredNames', how='inner')
    
    # Si hay coincidencias, agregar las columnas necesarias a matching_rows
    if not merged_data.empty:
        matching_rows = pd.concat([matching_rows, merged_data[['User_input']]])

# Renombrar la columna 'User_input' a 'Protein_ID'
matching_rows = matching_rows.rename(columns={'User_input': 'Protein_ID'})

# Guardar el archivo final
matching_rows.to_csv("ShinyGo/Patogeno/generate_TcruziSTRINGdb/PatogenoBase_TcruziSTRINGdb.tsv", sep='\t', index=False)

# Merged *T. cruzi*

In [39]:
# Cargar el archivo CSV con manejo de errores
def cargar_csv(ruta):
    try:
        df = pd.read_csv(ruta, sep='\t', header=None)  # Leer archivos .tsv
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo.")
        return None

# Guardar los archivos combinados en un solo archivo
def guardar_archivo_combinado(df1, df2, salida):
    if df1 is not None and df2 is not None:
        # Concatenar las dos columnas en un solo DataFrame
        df_combinado = pd.concat([df1, df2], ignore_index=True)
        df_combinado.to_csv(salida, index=False, header=False, sep='\t')
    else:
        print("No se pudo guardar el archivo porque hubo un error al cargar los datos.")

# Rutas de los archivos de entrada
ruta_tcruzi = "ShinyGo/Patogeno/generate_TcruziSTRINGdb/PatogenoBase_TcruziSTRINGdb.tsv"
ruta_tcruzistrainCLBrener = "ShinyGo/Patogeno/generate_TcruzistrainCLBrenerSTRINGdb/PatogenoBase_TcruzistrainCLBrenerSTRINGdb.tsv"

# Ruta del archivo de salida
salida_tsv = "ShinyGo/PatogenoBase_Tcruzi.tsv"

# Cargar los archivos
df_tcruzi = cargar_csv(ruta_tcruzi)
df_tcruzistrainCLBrener = cargar_csv(ruta_tcruzistrainCLBrener)

# Guardar el archivo combinado
guardar_archivo_combinado(df_tcruzi, df_tcruzistrainCLBrener, salida_tsv)

In [40]:
# Leer el archivo .tsv
tsv_file = "ShinyGo/PatogenoBase_Tcruzi.tsv"
df_tsv = pd.read_csv(tsv_file, sep='\t')

# Extraer la primera columna, convertirla a un set para eliminar duplicados
protein_set = set(df_tsv.iloc[:, 0])

# Guardar el resultado sin nombre de columna
protein_df = pd.DataFrame(list(protein_set))
protein_df.to_csv("ShinyGo/PatogenoBase_Tcruzi_SinProteinID.tsv", sep='\t', index=False, header=False)